In [ ]:
import featuretools as ft
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from pandas.tseries.offsets import *

## Process Data with DFS

In [ ]:
df_train = pd.read_csv('DataSet/TrainSet.csv')
df_val = pd.read_csv('DataSet/ValSet.csv')

In [ ]:
play_data = ['indicator4','indicator88','indicator2','indicator83','indicator75','midPrice',  'LastPrice', 'Volume', 'LastVolume', 'Turnover', 'LastTurnover',
       'OpenInterest', 'UpperLimitPrice', 'LowerLimitPrice']

In [ ]:
change_index = (df_train['am_pm'].values[:-1].astype(int) - df_train['am_pm'].values[1:].astype(int) != 0).nonzero()[0] + 1

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_train[play_data[i]+'_mean'] = np.zeros(df_train.shape[0])
    tmp = df_train[play_data[i]].values
    for j in range(30):
        df_train[play_data[i]+'_mean'] = df_train[play_data[i] + '_mean'] + tmp
        tmp = np.append([0] ,tmp[0:-1])
    sh = np.ones(df_train.shape[0]) * 30
    for j in range(len(change_index)):
        for k in range(30):
            sh[change_index[j] + k] = sh[change_index[j] + k] - 29 + k
            for l in range(30 - k):
                df_train[play_data[i]+'_mean'][change_index[j] + k] = df_train[play_data[i]+'_mean'][change_index[j] + k] - df_train[play_data[i]][change_index[j] - l - 1]         
    for j in range(30):
        sh[j] = j + 1
    df_train[play_data[i]+'_mean'] = df_train[play_data[i]+'_mean'] / sh

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_train[play_data[i]+'_std'] = np.zeros(df_train.shape[0])
    tmp = df_train[play_data[i]].values 
    for j in range(30):
        df_train[play_data[i]+'_std'] = df_train[play_data[i] + '_std'] + (tmp - df_train[play_data[i]+'_mean'].values) ** 2
        tmp = np.append([0] ,tmp[0:-1])
    sh = np.ones(df_train.shape[0]) * 30
    for j in range(len(change_index)):
        for k in range(30):
            sh[change_index[j] + k] = sh[change_index[j] + k] - 29 + k
            for l in range(30 - k):
                df_train[play_data[i]+'_std'][change_index[j] + k] = df_train[play_data[i]+'_std'][change_index[j] + k] - (df_train[play_data[i]][change_index[j] - l - 1] - df_train[play_data[i]+'_mean'][change_index[j] + k]) ** 2          
    for j in range(30):
        for k in range(30 - j - 1):
            df_train[play_data[i]+'_std'][j] = df_train[play_data[i]+'_std'][j] - (df_train[play_data[i]+'_mean'].values[j]) ** 2
        sh[j] = j + 1
    df_train[play_data[i]+'_std'] = df_train[play_data[i]+'_std'] / sh

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_train[play_data[i]+'_diff'] = df_train[play_data[i]]
    tmp = np.append([tmp[0]] ,tmp[0:-1])
    df_train[play_data[i]+'_diff'] = df_train[play_data[i]+'_diff'] - tmp
    for j in range(len(change_index)):
        df_train[play_data[i]][change_index[j]] = 0

In [ ]:
df_train.to_csv('DataSet/Dataset_DFSn.csv', index=False)

In [ ]:
change_index = (df_val['am_pm'].values[:-1].astype(int) - df_val['am_pm'].values[1:].astype(int) != 0).nonzero()[0] + 1

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_val[play_data[i]+'_mean'] = np.zeros(df_val.shape[0])
    tmp = df_val[play_data[i]].values
    for j in range(30):
        df_val[play_data[i]+'_mean'] = df_val[play_data[i] + '_mean'] + tmp
        tmp = np.append([0] ,tmp[0:-1])
    sh = np.ones(df_val.shape[0]) * 30
    for j in range(len(change_index)):
        for k in range(30):
            sh[change_index[j] + k] = sh[change_index[j] + k] - 29 + k
            for l in range(30 - k - 1):
                df_val[play_data[i]+'_mean'][change_index[j] + k] = df_val[play_data[i]+'_mean'][change_index[j] + k] - df_val[play_data[i]][change_index[j] - l - 1]         
    for j in range(30):
        sh[j] = j + 1
    df_val[play_data[i]+'_mean'] = df_val[play_data[i]+'_mean'] / sh

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_val[play_data[i]+'_std'] = np.zeros(df_val.shape[0])
    tmp = df_val[play_data[i]].values 
    for j in range(30):
        df_val[play_data[i]+'_std'] = df_val[play_data[i] + '_std'] + (tmp - df_val[play_data[i]+'_mean'].values) ** 2
        tmp = np.append([0] ,tmp[0:-1])
    sh = np.ones(df_val.shape[0]) * 30
    for j in range(len(change_index)):
        for k in range(30):
            sh[change_index[j] + k] = sh[change_index[j] + k] - 29 + k
            for l in range(30 - k - 1):
                df_val[play_data[i]+'_std'][change_index[j] + k] = df_val[play_data[i]+'_std'][change_index[j] + k] - (df_val[play_data[i]][change_index[j] - l - 1] - df_train[play_data[i]+'_mean'][change_index[j] + k]) ** 2          
    for j in range(30):
        for k in range(30 - j - 1):
            df_val[play_data[i]+'_std'][j] = df_val[play_data[i]+'_std'][j] - (df_val[play_data[i]+'_mean'].values[j]) ** 2
        sh[j] = j + 1
    df_val[play_data[i]+'_std'] = df_val[play_data[i]+'_std'] / sh

In [ ]:
for i in tqdm.tqdm(range(len(play_data))):
    df_val[play_data[i]+'_diff'] = df_val[play_data[i]]
    tmp = np.append([tmp[0]] ,tmp[0:-1])
    df_val[play_data[i]+'_diff'] = df_val[play_data[i]+'_diff'] - tmp
    for j in range(len(change_index)):
        df_val[play_data[i]][change_index[j]] = 0

In [ ]:
df_val.to_csv('DataSet/Dataset_DFSn_val.csv', index=False)

## Process Data with DWT

In [1]:
import pywt
def wavelet_transform(data,wavelet='db2',mode='reflect'):
    (cA,cD) = pywt.dwt(data,wavelet,mode)
    return pywt.upcoef('a',cA,wavelet,take = len(data)), \
            pywt.upcoef('d',cD,wavelet,take = len(data))

In [ ]:
df_train = pd.read_csv('DataSet/TrainSet.csv')
df_val = pd.read_csv('DataSet/ValSet.csv')
indicators = df_train.columns.values[:108].tolist()
market_stat = ['midPrice',  'LastPrice', 'Volume', 'LastVolume', 'Turnover', 'LastTurnover',
       'OpenInterest', 'UpperLimitPrice', 'LowerLimitPrice',] # 'am_pm',
#        'UpdateMinute']
features = indicators + market_stat
train_data = df_train[features]
train_data=train_data.values
val_data = df_val[features]
val_data=val_data.values


In [ ]:
from tqdm import tqdm
train_data_WT = np.empty((2,*train_data.shape))
for j in tqdm(range(len(train_data[0]))):
    A, D = pywt.wavelet_transform(train_data[:,j])
    train_data_WT[0,:,j] = A[:]
    train_data_WT[1,:,j] = D[:]

In [ ]:
val_data_WT = np.empty((2,*val_data.shape))
for j in tqdm(range(len(val_data[0]))):
    A, D = pywt.wavelet_transform(val_data[:,j])
    val_data_WT[0,:,j] = A[:]
    val_data_WT[1,:,j] = D[:]

In [ ]:
train_data_WT_merge = np.concatenate((train_data_WT[0],train_data_WT[1]),1)
val_data_WT_merge = np.concatenate((val_data_WT[0],val_data_WT[1]),1)